In [ ]:
import cv2

src = cv2.imread("building.jpg", cv2.IMREAD_GRAYSCALE)

dst = cv2.Canny(src, 50, 150)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np

src = cv2.imread("building.jpg", cv2.IMREAD_GRAYSCALE)

edges = cv2.Canny(src, 50, 150)

lines = cv2.HoughLinesP(edges, 1.0, np.pi / 180.0, 160, minLineLength=50, maxLineGap=5)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

if lines is not None:
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])
        pt2 = (lines[i][0][2], lines[i][0][3])
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import sys
import cv2
import numpy as np

src = cv2.imread("coins1.jpg")

if src is None:
    print("Image open failed!")
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)  # sigma = 1.0


def on_trackbar(pos):
    rmin = cv2.getTrackbarPos("minRadius", "img")
    rmax = cv2.getTrackbarPos("maxRadius", "img")
    th = cv2.getTrackbarPos("threshold", "img")

    circles = cv2.HoughCircles(
        blr,
        cv2.HOUGH_GRADIENT,
        1,
        50,
        param1=120,
        param2=th,
        minRadius=rmin,
        maxRadius=rmax,
    )

    dst = src.copy()
    sum_of_money = 0

    if circles is not None:
        for i in range(circles.shape[1]):
            cx, cy, radius = np.uint16(circles[0][i])
            cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)

            x1 = int(cx - radius)
            y1 = int(cy - radius)
            x2 = int(cx + radius)
            y2 = int(cy + radius)
            radius = int(radius)

            crop = dst[y1:y2, x1:x2, :]
            ch, cw = crop.shape[:2]

            mask = np.zeros((ch, cw), dtype=np.uint8)
            cv2.circle(mask, (cw // 2, ch // 2), radius, 255, -1)

            hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
            hue, _, _ = cv2.split(hsv)
            # hue_shift = (hue + 40) % 180
            mean_of_hue = cv2.mean(hue, mask)[0]

            won = 100
            if mean_of_hue < 50:
                won = 10

            sum_of_money += won

            desc = f"{sum_of_money} won"

            cv2.putText(
                crop,
                str(won),
                (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.0,
                255,
                2,
                cv2.LINE_AA,
            )

    cv2.putText(dst, desc, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, 255, 2, cv2.LINE_AA)
    cv2.imshow("img", dst)


cv2.imshow("img", src)

cv2.createTrackbar("minRadius", "img", 0, 100, on_trackbar)
cv2.createTrackbar("maxRadius", "img", 0, 150, on_trackbar)
cv2.createTrackbar("threshold", "img", 1, 100, on_trackbar)
cv2.setTrackbarPos("minRadius", "img", 10)
cv2.setTrackbarPos("maxRadius", "img", 80)
cv2.setTrackbarPos("threshold", "img", 40)
cv2.waitKey()

cv2.destroyAllWindows()

C:\Users\UserK\AppData\Local\Temp\ipykernel_6676\3812647481.py:40: RuntimeWarning: overflow encountered in scalar subtract
  x1 = int(cx - radius)
C:\Users\UserK\AppData\Local\Temp\ipykernel_6676\3812647481.py:41: RuntimeWarning: overflow encountered in scalar subtract
  y1 = int(cy - radius)


In [ ]:
import sys
import cv2
import numpy as np

src = cv2.imread("coins1.jpg")

if src is None:
    print("Image open failed!")
    sys.exit()

gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1.0)  # sigma = 1.0

# 원본 이미지의 높이, 너비 저장
h_orig, w_orig = src.shape[:2]


def on_trackbar(pos):
    rmin = cv2.getTrackbarPos("minRadius", "img")
    rmax = cv2.getTrackbarPos("maxRadius", "img")
    th = cv2.getTrackbarPos("threshold", "img")

    # HoughCircles 파라미터 유효성 검사 (rmin <= rmax)
    if rmin == 0:
        rmin = 1  # 최소 반지름 0 방지
    if rmin > rmax:
        print(f"Warning: minRadius({rmin}) > maxRadius({rmax}). Skipping HoughCircles.")
        # 또는 rmax를 rmin 값으로 설정: rmax = rmin
        # 또는 트랙바 값 강제 조정: cv2.setTrackbarPos("maxRadius", "img", rmin)
        # 여기서는 단순히 처리를 건너뛰도록 함
        dst = src.copy()  # 원본 복사본을 보여줌
        cv2.imshow("img", dst)
        return  # 함수 종료

    circles = cv2.HoughCircles(
        blr,
        cv2.HOUGH_GRADIENT,
        1,
        50,
        param1=120,
        param2=th,
        minRadius=rmin,
        maxRadius=rmax,
    )

    dst = src.copy()
    sum_of_money = 0
    desc = "0 won"  # 기본 설명 문자열 초기화

    if circles is not None:
        circles = np.uint16(
            np.around(circles)
        )  # 원 중심 및 반지름 정수화 (여기서는 해도 무방)
        for i in range(circles.shape[1]):
            # cx, cy, radius를 일반 int로 변환하여 사용
            cx, cy, radius = map(int, circles[0][i])

            # 원 그리기 (dst에 그림)
            cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)

            # --- 좌표 계산 및 경계 검사 ---
            x1 = cx - radius
            y1 = cy - radius
            x2 = cx + radius
            y2 = cy + radius

            # 좌표가 이미지 경계 내에 있는지 확인 및 조정
            x1_clipped = max(0, x1)
            y1_clipped = max(0, y1)
            x2_clipped = min(w_orig, x2)
            y2_clipped = min(h_orig, y2)

            # 조정된 좌표로 crop 영역이 유효한지(넓이/높이가 0보다 큰지) 확인
            if x1_clipped >= x2_clipped or y1_clipped >= y2_clipped:
                print(
                    f"Warning: Invalid crop area for circle at ({cx},{cy}) R={radius}. Skipping."
                )
                continue  # 다음 원으로 넘어감

            # 원본 이미지(src)에서 crop (dst는 매번 새로 복사되므로)
            # crop = dst[y1_clipped:y2_clipped, x1_clipped:x2_clipped] # 이렇게 해도 됨
            crop = src[y1_clipped:y2_clipped, x1_clipped:x2_clipped]

            # crop된 이미지 유효성 재확인 (매우 드문 경우 대비)
            if crop.size == 0:
                print(
                    f"Warning: Empty crop for circle at ({cx},{cy}) R={radius} after clipping. Skipping."
                )
                continue

            ch, cw = crop.shape[:2]

            # --- 마스크 생성 및 색상 분석 ---
            # 마스크는 crop된 이미지 크기에 맞게 생성
            mask = np.zeros((ch, cw), dtype=np.uint8)

            # 마스크의 중심 좌표 계산 (crop된 이미지 기준)
            # 원의 중심이 crop 영역 내부에 어떻게 위치하는지 계산해야 함
            mask_center_x = cx - x1_clipped
            mask_center_y = cy - y1_clipped

            # 마스크에 원 그리기 (원래 반지름 사용)
            cv2.circle(mask, (mask_center_x, mask_center_y), radius, 255, -1)

            # 색상 변환 및 평균 색상 계산
            hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
            hue, _, _ = cv2.split(hsv)
            # hue_shift = (hue + 40) % 180
            mean_of_hue = cv2.mean(hue, mask)[0]  # 마스크 영역 내 평균 Hue

            # 동전 종류 판별 (Hue 값 기준)
            won = 100
            if mean_of_hue < 50:  # 이 값은 이미지와 동전 종류에 따라 조정 필요
                won = 10

            sum_of_money += won

            # --- 결과 텍스트 표시 ---
            # crop된 이미지 위에 금액 표시 (좌표는 crop 기준)
            # cv2.putText(crop,...) 대신 dst에 직접 쓰는 것이 더 관리하기 편할 수 있음
            # 아래는 dst에 텍스트를 쓰는 방식으로 변경
            text_pos_x = cx - radius // 2  # 원 중심 근처에 표시
            text_pos_y = cy
            cv2.putText(
                dst,  # crop 대신 dst에 직접 씀
                str(won),
                (text_pos_x, text_pos_y),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.8,  # 폰트 크기 약간 줄임
                (0, 255, 0),  # 텍스트 색상 변경 (초록색)
                2,
                cv2.LINE_AA,
            )

        # 최종 합계 금액 업데이트
        desc = f"{sum_of_money} won"

    # 최종 결과 이미지에 총 금액 표시
    cv2.putText(
        dst,
        desc,
        (10, 40),
        cv2.FONT_HERSHEY_SIMPLEX,
        1.0,
        (255, 255, 255),
        2,
        cv2.LINE_AA,
    )  # 흰색 텍스트
    cv2.imshow("img", dst)


# --- 메인 코드 ---
cv2.imshow("img", src)  # 초기 이미지 표시

# 트랙바 생성 (콜백 함수 지정)
cv2.createTrackbar("minRadius", "img", 0, 100, on_trackbar)
cv2.createTrackbar("maxRadius", "img", 0, 150, on_trackbar)
cv2.createTrackbar("threshold", "img", 1, 100, on_trackbar)

# 초기 트랙바 값 설정 (이때 on_trackbar가 호출됨)
cv2.setTrackbarPos("minRadius", "img", 10)
cv2.setTrackbarPos("maxRadius", "img", 80)
cv2.setTrackbarPos("threshold", "img", 40)

# 초기 호출을 위해 수동으로 한 번 호출 (setTrackbarPos가 호출하므로 필요 없을 수 있음)
# on_trackbar(0) # 주석 처리하거나 제거 가능

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)

# 전역 이진화 (OTSU)
_, dst1 = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 지역 이진화 (OTSU)
# 4x4 블록으로 나누어 이진화
dst2 = np.zeros(src.shape, dtype=np.uint8)

bw = src.shape[1] // 4
bh = src.shape[0] // 4

for y in range(4):
    for x in range(4):
        src_ = src[y * bh : (y + 1) * bh, x * bw : (x + 1) * bw]
        dst_ = dst2[y * bh : (y + 1) * bh, x * bw : (x + 1) * bw]
        cv2.threshold(src_, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU, dst_)


cv2.imshow("src", src)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.waitKey()
cv2.destroyAllWindows()

In [2]:
import cv2

src = cv2.imread("sudoku.jpg", cv2.IMREAD_GRAYSCALE)


def on_trackbar(pos):
    bsize = pos
    if bsize % 2 == 0:
        bsize -= 1
    if bsize < 3:
        bsize = 3

    dst = cv2.adaptiveThreshold(
        src, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, bsize, 5
    )

    cv2.imshow("dst", dst)


cv2.imshow("src", src)
cv2.namedWindow("dst")
cv2.createTrackbar("block size", "dst", 0, 200, on_trackbar)
cv2.setTrackbarPos("block size", "dst", 11)

cv2.waitKey()
cv2.destroyAllWindows()


In [6]:
import cv2
import numpy as np

src = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)

# 지역 이진화 (OTSU)
# 4x4 블록으로 나누어 이진화
dst1 = np.zeros(src.shape, dtype=np.uint8)

bw = src.shape[1] // 4
bh = src.shape[0] // 4

for y in range(4):
    for x in range(4):
        src_ = src[y * bh : (y + 1) * bh, x * bw : (x + 1) * bw]
        dst_ = dst1[y * bh : (y + 1) * bh, x * bw : (x + 1) * bw]
        cv2.threshold(src_, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU, dst_)

cnt1, _ = cv2.connectedComponents(dst1)
print(f"cnt1: {cnt1}")

dst2 = cv2.morphologyEx(dst1, cv2.MORPH_OPEN, None)
# dst2 = cr2.erode(dst1, None)
# dst2 = cv2.dilate(dst1, None)

cnt2, _ = cv2.connectedComponents(dst2)
print(f"cnt2: {cnt2}")

cv2.imshow("src", src)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.waitKey()
cv2.destroyAllWindows()

cnt1: 113
cnt2: 99


In [ ]:
src = cv2.imread("keyboard.bmp", cv2.IMREAD_GRAYSCALE)

_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU)
cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(src_bin)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for i in range(1, cnt):
    (x, y, w, h, area) = stats[i]

    if area < 20:
        continue

    cv2.rectangle(dst, (x, y, w, h), (0, 0, 255))

cv2.imshow("src", src)
cv2.imshow("src_bin", src_bin)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [2]:
import cv2
import random

src = cv2.imread("contours.bmp", cv2.IMREAD_GRAYSCALE)

# contours, hier = cv2.findContours(src, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
contours, hier = cv2.findContours(src, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

idx = 0
while idx >= 0:
    c = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    cv2.drawContours(dst, contours, idx, c, 2, cv2.LINE_8, hier)
    idx = hier[0][idx][0]

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [15]:
import cv2
import numpy as np

src = cv2.imread("milkdrop.bmp", cv2.IMREAD_GRAYSCALE)

_, src_bin = cv2.threshold(src, 0, 255, cv2.THRESH_OTSU)
contours, hier = cv2.findContours(src_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

h, w = src.shape[:2]
dst = np.zeros((h, w, 3), np.uint8)

for i in range(len(contours)):
    c = (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255),
    )
    cv2.drawContours(dst, contours, i, c, 1)

cv2.imshow("src", src)
cv2.imshow("src_bin", src_bin)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [17]:
import cv2
import math


def setLabel(img, pts, label):
    (x, y, w, h) = cv2.boundingRect(pts)
    pt1 = (x, y)
    pt2 = (x + w, y + h)
    cv2.rectangle(img, pt1, pt2, (0, 0, 255), 1)
    cv2.putText(img, label, pt1, cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255))


img = cv2.imread("polygon.bmp", cv2.IMREAD_COLOR)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, img_bin = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
contours, hier = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for pts in contours:
    if cv2.contourArea(pts) < 400:
        continue

    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True) * 0.02, True)
    vtc = len(approx)

    if vtc == 3:
        setLabel(img, pts, "TRI")
    elif vtc == 4:
        setLabel(img, pts, "RECT")
    else:
        length = cv2.arcLength(pts, True)
        area = cv2.contourArea(pts)
        ratio = 4.0 * math.pi * area / (length * length)

        if ratio > 0.85:
            setLabel(img, pts, "CIR")

cv2.imshow("img", img)
cv2.imshow("img_bin", img_bin)
cv2.waitKey()
cv2.destroyAllWindows()

In [3]:
import cv2
import pytesseract

img = cv2.imread("sudoku.jpg")

img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print(pytesseract.image_to_string(img_gray))

In [24]:
import sys
import random
import numpy as np
import cv2
import pytesseract


def reorderPts(pts):
    idx = np.lexsort(
        (pts[:, 1], pts[:, 0])
    )  # 칼럼0 -> 칼럼1 순으로 정렬한 인덱스를 반환
    pts = pts[idx]  # x좌표로 정렬

    if pts[0, 1] > pts[1, 1]:
        pts[[0, 1]] = pts[[1, 0]]

    if pts[2, 1] < pts[3, 1]:
        pts[[2, 3]] = pts[[3, 2]]

    return pts


# 영상 불러오기
file_path = r"C:\Users\UserK\projects\Python\20250423\namecard2.jpg"

src = cv2.imread(file_path)

if src is None:
    print("Image load failed!")
    sys.exit()

# 출력 영상 설정
dw, dh = 720, 400
srcQuad = np.array([[0, 0], [0, 0], [0, 0], [0, 0]], np.float32)
dstQuad = np.array([[0, 0], [0, dh], [dw, dh], [dw, 0]], np.float32)
dst = np.zeros((dh, dw), np.uint8)

# 입력 영상 전처리
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
_, src_bin = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# 외곽선 검출 및 명함 검출
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cpy = src.copy()
for pts in contours:
    # 너무 작은 객체는 무시
    if cv2.contourArea(pts) < 1000:
        continue

    # 외곽선 근사화
    approx = cv2.approxPolyDP(pts, cv2.arcLength(pts, True) * 0.02, True)

    # 컨벡스가 아니고, 사각형이 아니면 무시
    if not cv2.isContourConvex(approx) or len(approx) != 4:
        continue

    cv2.polylines(cpy, [approx], True, (0, 255, 0), 2, cv2.LINE_AA)
    srcQuad = reorderPts(approx.reshape(4, 2).astype(np.float32))

pers = cv2.getPerspectiveTransform(srcQuad, dstQuad)
dst = cv2.warpPerspective(src, pers, (dw, dh))

dst_gray = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
print(pytesseract.image_to_string(dst_gray, lang="Hangul+eng"))

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()


황선 선 규 (Sunkyoo Hwang)

17 전문서 저자 / 17 강사 / 공 학 박사

#OpenCV #Computer Vision
#Deep Leaming #Algorithm

sunkyoo.hwang@gmail.com
https://sunkyoo.github.io/



In [31]:
src = cv2.imread("nemo.jpg")

rc = cv2.selectROI(src)
mask = np.zeros(src.shape[:2], np.uint8)

cv2.grabCut(src, mask, rc, None, None, 5, cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")

dsc = src * mask2[:, :, np.newaxis]

cv2.imshow("mask", mask2 * 255)
cv2.imshow("dst", dsc)
cv2.waitKey()
cv2.destroyAllWindows()

In [36]:
import cv2
import numpy as np

src = cv2.imread("messi5.jpg")

rc = cv2.selectROI(src)
mask = np.zeros(src.shape[:2], np.uint8)

bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)

mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
dst = src * mask2[:, :, np.newaxis]

cv2.imshow("dst", dst)


def on_mouse(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)
        cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
        cv2.imshow("dst", dst)
    elif event == cv2.EVENT_RBUTTONDOWN:
        cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)
        cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
        cv2.imshow("dst", dst)
    elif event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(dst, (x, y), 3, (255, 0, 0), -1)
            cv2.circle(mask, (x, y), 3, cv2.GC_FGD, -1)
            cv2.imshow("dst", dst)
        elif flags & cv2.EVENT_FLAG_RBUTTON:
            cv2.circle(dst, (x, y), 3, (0, 0, 255), -1)
            cv2.circle(mask, (x, y), 3, cv2.GC_BGD, -1)
            cv2.imshow("dst", dst)


cv2.setMouseCallback("dst", on_mouse)

while True:
    key = cv2.waitKey()
    if key == 13:  # Enter
        cv2.grabCut(src, mask, rc, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_MASK)
        mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
        dsc = src * mask2[:, :, np.newaxis]
        cv2.imshow("dst", dsc)
    elif key == 27:  # ESC
        break

cv2.destroyAllWindows()

In [44]:
obj = cv2.imread("spades.png", cv2.IMREAD_GRAYSCALE)
src = cv2.imread("symbols.png", cv2.IMREAD_GRAYSCALE)

_, obj_bin = cv2.threshold(obj, 128, 255, cv2.THRESH_BINARY_INV)
obj_contours, _ = cv2.findContours(obj_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
obj_pts = obj_contours[0]

_, src_bin = cv2.threshold(src, 128, 255, cv2.THRESH_BINARY_INV)
contours, _ = cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for pts in contours:
    rc = cv2.boundingRect(pts)
    cv2.rectangle(dst, rc, (255, 0, 0), 1)

    dist = cv2.matchShapes(obj_pts, pts, cv2.CONTOURS_MATCH_I3, 0)

    cv2.putText(
        dst,
        str(round(dist, 4)),
        (rc[0], rc[1] - 3),
        cv2.FONT_HERSHEY_COMPLEX,
        0.6,
        (255, 0, 0),
        1,
        cv2.LINE_AA,
    )

    if dist < 0.1:
        cv2.rectangle(dst, rc, (0, 0, 255), 2)

cv2.imshow("src", src)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
src = cv2.imread("circuit.bmp", cv2.IMREAD_GRAYSCALE)
templ = cv2.imread("crystal.bmp", cv2.IMREAD_GRAYSCALE)

noise = np.zeros(src.shape, np.int32)
cv2.randn(noise, 50, 10)
src = cv2.add(src, noise, dtype=cv2.CV_8UC3)

res = cv2.matchTemplate(src, templ, cv2.TM_CCOEFF_NORMED)
res_norm = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

_, maxv, _, maxloc = cv2.minMaxLoc(res)
print(f"maxv: {maxv}")
print(f"maxloc: {maxloc}")

th, tw = templ.shape[:2]
dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
cv2.rectangle(dst, maxloc, (maxloc[0] + tw, maxloc[1] + th), (0, 0, 255), 2)

cv2.imshow("src", src)
cv2.imshow("res_norm", res_norm)
cv2.imshow("dst", dst)
cv2.waitKey()
cv2.destroyAllWindows()

maxv: 0.9802818298339844
maxloc: (568, 320)


In [ ]:
def load_digits():
    img_digits = []

    for i in range(10):
        filename = f"./digits/digit{i}.bmp"
        img_digits.append(cv2.imread(filename, cv2.IMREAD_GRAYSCALE))

        if img_digits[i] is None:
            return None

    return img_digits


def find_digit(img, img_digits):
    max_idx = -1
    max_ccoeff = -1

    for i in range(10):
        img = cv2.resize(img, (100, 150))
        res = cv2.matchTemplate(img, img_digits[i], cv2.TM_CCOEFF_NORMED)

        if res[0, 0] > max_ccoeff:
            max_idx = i
            max_ccoeff = res[0, 0]

    return max_idx


def main():
    src = cv2.imread("digits_print.bmp")

    if src is None:
        print("Image load failed!")
        return

    img_digits = load_digits()

    if img_digits is None:
        print("Digit image load failed!")
        return

    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, src_bin = cv2.threshold(
        src_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU
    )
    cnt, _, stats, _ = cv2.connectedComponentsWithStats(src_bin)

    dst = src.copy()

    for i in range(1, cnt):
        (x, y, w, h, s) = stats[i]

        if s < 1000:
            continue

        digit = find_digit(src_gray[y : y + h, x : x + w], img_digits)
        cv2.rectangle(dst, (x, y, w, h), (0, 255, 255))
        cv2.putText(
            dst,
            str(digit),
            (x, y - 4),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 255, 255),
            2,
            cv2.LINE_AA,
        )

    cv2.imshow("dst", dst)
    cv2.waitKey()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()

In [64]:
src = cv2.imread(r"C:\Users\UserK\projects\Python\20250422\lenna.bmp")

classifier = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")

faces = classifier.detectMultiScale(src)

for (x, y, w, h) in faces:
    cv2.rectangle(src, (x, y, w, h,), (255, 0, 255), 2)

cv2.imshow("src", src)
cv2.waitKey()
cv2.destroyAllWindows()